In [1]:
from torch.nn.utils import prune
from copy import deepcopy
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

In [5]:
fc_demo = nn.Linear(10, 10)

module = deepcopy(fc_demo)
module.weight

Parameter containing:
tensor([[-0.2068,  0.1001, -0.1947, -0.0021,  0.0076,  0.0999,  0.0389, -0.2125,
         -0.0847, -0.2272],
        [-0.1763, -0.2126, -0.1043,  0.0072, -0.0704, -0.0482,  0.1120,  0.0584,
         -0.0800, -0.2275],
        [ 0.2191, -0.1361, -0.1143,  0.0951,  0.2873, -0.0298,  0.0353,  0.2329,
         -0.1792, -0.2830],
        [-0.3015,  0.1880, -0.1756, -0.0083,  0.2839, -0.1547, -0.2182, -0.2378,
         -0.2123, -0.0172],
        [-0.2185,  0.0745,  0.0259,  0.2885,  0.1285,  0.2599,  0.1201, -0.2467,
          0.2847,  0.0434],
        [-0.1655,  0.1090,  0.1061, -0.3135, -0.2688, -0.2015,  0.0928,  0.1609,
          0.0142, -0.1683],
        [-0.0152,  0.1624, -0.1609, -0.2250, -0.2882, -0.2305, -0.1809,  0.0763,
          0.1467, -0.2712],
        [-0.0383,  0.1933,  0.0670, -0.1918,  0.2453,  0.1982, -0.1134,  0.1842,
          0.2441,  0.2082],
        [ 0.2372, -0.2610,  0.0997,  0.0213,  0.1941,  0.1666, -0.2475, -0.0959,
         -0.3003,  0.1177

# L1 Unstructured

## Numpy

In [7]:
weight = module.weight.detach().cpu().numpy()
print(weight)

[[-0.2067636   0.10005851 -0.19469348 -0.00207557  0.00760038  0.09990957
   0.03889052 -0.21254541 -0.08468141 -0.2272342 ]
 [-0.17631647 -0.21260414 -0.10425305  0.00717756 -0.07036395 -0.04820985
   0.11202239  0.05839178 -0.08002177 -0.22754765]
 [ 0.21913832 -0.136076   -0.11425573  0.09511512  0.28728288 -0.02980053
   0.0352804   0.23286301 -0.17921785 -0.28302234]
 [-0.3015032   0.18798684 -0.1756044  -0.00826513  0.28386784 -0.15472837
  -0.2181603  -0.23783697 -0.21226339 -0.01715389]
 [-0.21848646  0.07452309  0.02591428  0.28845793  0.12847835  0.25985664
   0.12008414 -0.24674857  0.2847134   0.04337227]
 [-0.165492    0.10899416  0.10610964 -0.31352687 -0.2688433  -0.20150705
   0.0928008   0.16090575  0.01419759 -0.16826236]
 [-0.01516181  0.16243987 -0.16087332 -0.22498997 -0.28822803 -0.23045282
  -0.18093438  0.07628943  0.14666623 -0.2712112 ]
 [-0.03825615  0.19325763  0.06701797 -0.19183524  0.24532376  0.19821037
  -0.1133611   0.18417628  0.24409811  0.20822899]


In [9]:
sorted_array = np.sort(np.abs(weight.ravel()))
print(sorted_array)

[0.00207557 0.00717756 0.00760038 0.00826513 0.01419759 0.01516181
 0.01715389 0.02072657 0.02132404 0.02591428 0.02980053 0.0352804
 0.03825615 0.03889052 0.04135095 0.04337227 0.04820985 0.05538459
 0.05839178 0.06701797 0.07036395 0.07452309 0.07628943 0.08002177
 0.08468141 0.0928008  0.09511512 0.09587992 0.09970548 0.09990957
 0.10005851 0.10425305 0.10610964 0.10668495 0.10899416 0.11202239
 0.1133611  0.11425573 0.11768264 0.12008414 0.12847835 0.136076
 0.14666623 0.15435301 0.15472837 0.160057   0.16087332 0.16090575
 0.16243987 0.165492   0.16660728 0.16826236 0.1756044  0.17631647
 0.17921785 0.18093438 0.18417628 0.18798684 0.19183524 0.19325763
 0.19405255 0.19469348 0.19531356 0.19821037 0.20150705 0.2032623
 0.2067636  0.20822899 0.21226339 0.21254541 0.21260414 0.2181603
 0.21848646 0.21913832 0.22498997 0.2272342  0.22754765 0.23045282
 0.2325601  0.23286301 0.23721644 0.23783697 0.24409811 0.24532376
 0.24674857 0.24749333 0.25985664 0.26099268 0.26267338 0.2688433
 

In [12]:
mask = np.abs(weight) > sorted_array[29]
print(mask.astype(int))

[[1 1 1 0 0 0 0 1 0 1]
 [1 1 1 0 0 0 1 0 0 1]
 [1 1 1 0 1 0 0 1 1 1]
 [1 1 1 0 1 1 1 1 1 0]
 [1 0 0 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 0 1 0 1]
 [0 1 1 1 1 1 1 0 1 1]
 [0 1 0 1 1 1 1 1 1 1]
 [1 1 0 0 1 1 1 0 1 1]
 [1 1 0 1 0 1 1 1 0 1]]


## Pytorch

In [15]:
prune.l1_unstructured(module, 'weight', amount=0.3)

Linear(in_features=10, out_features=10, bias=True)

In [16]:
weight = module.weight.detach().cpu().numpy()
weight

array([[-0.2067636 ,  0.10005851, -0.19469348, -0.        ,  0.        ,
         0.        ,  0.        , -0.21254541, -0.        , -0.2272342 ],
       [-0.17631647, -0.21260414, -0.10425305,  0.        , -0.        ,
        -0.        ,  0.11202239,  0.        , -0.        , -0.22754765],
       [ 0.21913832, -0.136076  , -0.11425573,  0.        ,  0.28728288,
        -0.        ,  0.        ,  0.23286301, -0.17921785, -0.28302234],
       [-0.3015032 ,  0.18798684, -0.1756044 , -0.        ,  0.28386784,
        -0.15472837, -0.2181603 , -0.23783697, -0.21226339, -0.        ],
       [-0.21848646,  0.        ,  0.        ,  0.28845793,  0.12847835,
         0.25985664,  0.12008414, -0.24674857,  0.2847134 ,  0.        ],
       [-0.165492  ,  0.10899416,  0.10610964, -0.31352687, -0.2688433 ,
        -0.20150705,  0.        ,  0.16090575,  0.        , -0.16826236],
       [-0.        ,  0.16243987, -0.16087332, -0.22498997, -0.28822803,
        -0.23045282, -0.18093438,  0.        

# L1 Structured

## Numpy

In [18]:
module = deepcopy(fc_demo)
weight = module.weight.detach().cpu().numpy()

print(weight)

[[-0.2067636   0.10005851 -0.19469348 -0.00207557  0.00760038  0.09990957
   0.03889052 -0.21254541 -0.08468141 -0.2272342 ]
 [-0.17631647 -0.21260414 -0.10425305  0.00717756 -0.07036395 -0.04820985
   0.11202239  0.05839178 -0.08002177 -0.22754765]
 [ 0.21913832 -0.136076   -0.11425573  0.09511512  0.28728288 -0.02980053
   0.0352804   0.23286301 -0.17921785 -0.28302234]
 [-0.3015032   0.18798684 -0.1756044  -0.00826513  0.28386784 -0.15472837
  -0.2181603  -0.23783697 -0.21226339 -0.01715389]
 [-0.21848646  0.07452309  0.02591428  0.28845793  0.12847835  0.25985664
   0.12008414 -0.24674857  0.2847134   0.04337227]
 [-0.165492    0.10899416  0.10610964 -0.31352687 -0.2688433  -0.20150705
   0.0928008   0.16090575  0.01419759 -0.16826236]
 [-0.01516181  0.16243987 -0.16087332 -0.22498997 -0.28822803 -0.23045282
  -0.18093438  0.07628943  0.14666623 -0.2712112 ]
 [-0.03825615  0.19325763  0.06701797 -0.19183524  0.24532376  0.19821037
  -0.1133611   0.18417628  0.24409811  0.20822899]


In [20]:
# L2 norm
print(np.sum( np.abs(weight**2), axis=1))
print(np.argsort(np.sum( np.abs(weight**2), axis=1)))

[0.20620678 0.16862354 0.3397496  0.41118062 0.37944964 0.324724
 0.37292507 0.32929015 0.37469074 0.26857615]
[1 0 9 5 7 2 6 8 4 3]


In [22]:
sorted_array = np.argsort(np.sum( np.abs(weight), axis=1))
print(sorted_array)

[1 0 9 5 2 7 4 8 6 3]


In [25]:
weight[sorted_array[:3], :] = 0
pruned_weight = deepcopy(weight)
print(pruned_weight)

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.21913832 -0.136076   -0.11425573  0.09511512  0.28728288 -0.02980053
   0.0352804   0.23286301 -0.17921785 -0.28302234]
 [-0.3015032   0.18798684 -0.1756044  -0.00826513  0.28386784 -0.15472837
  -0.2181603  -0.23783697 -0.21226339 -0.01715389]
 [-0.21848646  0.07452309  0.02591428  0.28845793  0.12847835  0.25985664
   0.12008414 -0.24674857  0.2847134   0.04337227]
 [-0.165492    0.10899416  0.10610964 -0.31352687 -0.2688433  -0.20150705
   0.0928008   0.16090575  0.01419759 -0.16826236]
 [-0.01516181  0.16243987 -0.16087332 -0.22498997 -0.28822803 -0.23045282
  -0.18093438  0.07628943  0.14666623 -0.2712112 ]
 [-0.03825615  0.19325763  0.06701797 -0.19183524  0.24532376  0.19821037
  -0.1133611   0.18417628  0.24409811  0.20822899]
 [ 0.23721644 -0

## Pytorch

In [27]:
module = deepcopy(fc_demo)
prune.ln_structured(module, 'weight', n=2, amount=0.3, dim=0)

Linear(in_features=10, out_features=10, bias=True)

In [33]:
print(module.weight)
print(list(module.named_buffers()))

tensor([[-0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000,
         -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000,
         -0.0000, -0.0000],
        [ 0.2191, -0.1361, -0.1143,  0.0951,  0.2873, -0.0298,  0.0353,  0.2329,
         -0.1792, -0.2830],
        [-0.3015,  0.1880, -0.1756, -0.0083,  0.2839, -0.1547, -0.2182, -0.2378,
         -0.2123, -0.0172],
        [-0.2185,  0.0745,  0.0259,  0.2885,  0.1285,  0.2599,  0.1201, -0.2467,
          0.2847,  0.0434],
        [-0.1655,  0.1090,  0.1061, -0.3135, -0.2688, -0.2015,  0.0928,  0.1609,
          0.0142, -0.1683],
        [-0.0152,  0.1624, -0.1609, -0.2250, -0.2882, -0.2305, -0.1809,  0.0763,
          0.1467, -0.2712],
        [-0.0383,  0.1933,  0.0670, -0.1918,  0.2453,  0.1982, -0.1134,  0.1842,
          0.2441,  0.2082],
        [ 0.2372, -0.2610,  0.0997,  0.0213,  0.1941,  0.1666, -0.2475, -0.0959,
         -0.3003,  0.1177],
        [ 0.0000, -